# HtrA1 Crystal Simulation

## Relevant System info:

- **Temperature**: 100 K

- **pH**: 5.6
- **X-ray wavelength**: 1.08090 A
- **X-ray Resolution Range**: (50.0, 2.750) A
- **Matthews Coefficient, $V_m$**: 3.61 (A^3 / Da)
    - Note: this is the volume of the asymmetric unit, as obtained directly from the X-ray diffraction measurements, divided by the molecular weight of protein contained in the asymmetric unit -- i.e. the crystal volume per unit of molecular wieght
- **Solvent Content, $V_s$**: 65.89 % 
    - $V_s = 1 - \frac{1.23}{V_m}$
- **Crystallization Conditions**:
    - 1.0 M Li2SO4
    - 0.1 M Sodium Citrate (pH 5.6)
    - 0.5 M (NH4)2SO4
    - Temperature: 292 K
    - Vapor Diffusion
    - Sitting Drop
    - Temp: 292 K

## PDB Cleaning and Supercell Prep

## PDB File Preparation:
### Issues with protein connectivity

### 3NUM
Protein has three separate parts:
- Fragment 1: Resid 315 to 359
- Fragment 2: Resid 290 to 300
- Fragment 3: Resid 161 to 284

Using the program `pdbfixer` to fill in the missing residues, without the pH 7 hydrogens, and outputted to new pdb files `3num_fixed.pdb`

3NUM -- resid 161 to 284 and resid 290 to 300 and resid 315 to 369
(3TJN -- resid 164 to 367)
3NZI -- resid 160 to 370 (4-7)
3NWU -- resid 161 to 300 and resid 314 to 364 (protein and chain A)

![HtrA1 Overlay](./misc_files/HtrA1_overlay_all_three.png)

**Fig. 1**: 3NUM (red), 3NZI (blue) and 3NWU (green) overlaid, with loop region missing in 3NUM opage in the other two images.

Going to try using Chimera and modeller/modloop to attach better loop

![3NUM Fragments](./misc_files/3NUM_fragments.png)

**Fig. 2** -- 3NUM structure, with the three fragments in three different colors.

## 3NUM

3 fragments (`pfrags`):
- resid 161 to 284
- resid 290 to 300
- resid 315 to 369

![3NZI Fragments](./misc_files/3NZI_fragments.png)

**Fig. 3** -- 3NZI structure, with ligand in orange

## 3NZI

2 fragments (`pfrags`) and one ligand ("B2V": $\text{C}_{4}\text{H}_{12}\text{BNO}$)
- resid 160 to 370
- resid 1 to 7

## PDBFixer

Used PDBFixer to reconstruct the missing loops in `3NUM` residues 285 to 289 and 301 to 314, then kept only as much N- and C-terminal residues to match exactly with the N- and C-terminal ends of the original `3NUM` file. Also filled in heavy atoms but did not add hydrogens

## Chimera Tools -- Model/Refine Loops

### Outputted 5 models (#2.1-#2.5) with the following zDOPE scores

(reset relevant resids from 5 to 213 instead of 161 to 369)

**#2.1** -- -1.29

**#2.4** -- -1.28

**#2.3** -- -1.24

**#2.5** -- -1.22

**#2.2** -- -1.03

### Commands for fitting and outputting PDB structures in VMD:

(assuming 3num is molecule 0, 3num_with_loop is molecule 1)

```
set 3NUM_Fit [atomselect 0 "name CA and (resid 5 to 128 or resid 134 to 144 or resid 159 to 213)"]
set 3NUM_Loop_Fit [atomselect 1 "name CA and (resid 5 to 128 or resid 134 to 144 or resid 159 to 213)"]
set M [measure fit $3NUM_Loop_Fit $3NUMfit]
set 3NUM_Loop [atomselect 1 all]
$3NUM_Loop move $M
set 3NUM_out [atomselect 1 "resid 5 to 213"]
$3NUM_out writepdb "3num_loop_1_aligned.pdb"
```

### January 26th: 
- Aligned the 3num and 3num_fixed_chim_loop_1.pdb structures using the atomselection above, then outputted the coordinates to a PDB file, leaving off the first four residues, which were added by pdb fixer, and aren't present in 3num (3num_loop_1_aligned.pdb), and copied the header from 3num.pdb to the top of 3num_loop_1_aligned.pdb for scale and crystal information.

Now just need to check the bond orders and add hydrogens before putting in to UnitCell and PropPDB, and do the same for two other loop starting configuration structures.

#### Questions to ask later:

Once we see what starting configuration best matches with the diffuse data, is there any way for that information to be used to refine the crystallographic structure?

#### January 30th, 2018

Fixed `3num_loop_(#)_aligned.pdb` residue numbers with this script (practiced on a `_copy`):

#### `fix_residues.py`

```with open("3num_loop_3_aligned_new.pdb", 'r') as file:
    lines = file.readlines()
    with open("3num_loop_3_aligned_new2.pdb",'w') as newfile:
        for line in lines:
            if line[0:6] == 'ATOM  ':
                old_resid = line[23:26]
                new_resid = int(old_resid) + 156
                new_resid_str = str(new_resid)
                newline = line[:20] + ' A ' + new_resid_str + line[26:]
                newfile.write(newline)
            else:
                newfile.write(line)
        newfile.write('TER    1599      ASP A 369\nMASTER      560    0    0    3   14    0    0    6 1598    1    0   26\nEND')```

Now just have to make sure the header doesn't interfere with the necessary parts of the pdb file that parmed and openeye will need to fix the pdb and UnitCell and PropPDB will need to prepare them for simulation

In [ ]:
import nglview

view1 = nglview.show_file("/Users/davidwych/Downloads/GP3_Dir/PDB2PQR_Files/3num_loop_1_aligned_new_clean2.pdb")
view2 = nglview.show_file("/Users/davidwych/Downloads/GP3_Dir/PDB2PQR_Files/3num_loop_2_aligned_new_clean2.pdb")
view3 = nglview.show_file("/Users/davidwych/Downloads/GP3_Dir/PDB2PQR_Files/3num_loop_3_aligned_new_clean2.pdb")

for el in [view1, view2, view3]:
    el.clear_representations()
    el.add_representation(repr_type="cartoon", selection="300-315")
    el.add_representation(repr_type="cartoon", selection="161-284", opacity=0.1)
    el.add_representation(repr_type="cartoon", selection="285-289")
    el.add_representation(repr_type="cartoon", selection="290-299", opacity=0.1)
    el.add_representation(repr_type="cartoon", selection="315-369", opacity=0.1)

In [ ]:
from ipywidgets import VBox

### Loop Configurations

In [ ]:
vbox = VBox([view1, view2, view3])
vbox

In [ ]:
for view in [view1, view2, view3]:
    view.sync_view()

### PDB2PQR

Converted the aligned PDB files to PQR files using http://nbcr-222.ucsd.edu/pdb2pqr_2.0.0/, selecting AMBER Force Field and naming conventions, and choosing a pH of 7
- There was a problem with all three loop configurations, which is apparently a result of not being able to work around a clash between these two atoms: `HIS A 226 HA is too close to HIS A 226 CD2`, but it was unable to "debump" it. It came with this warning in the pqr file:
    - `WARNING: Unable to debump HIS A 226`

### Using `parmed` to convert the PQRs to PDBs

In [ ]:
import parmed as pmd

In [ ]:
pqr1 = pmd.load_file("./PDB2PQR_Files/3num_loop_1_aligned_new.pqr")
pqr1.save("./PDB2PQR_Files/3num_loop_1_aligned_new_clean.pdb")

pqr2 = pmd.load_file("./PDB2PQR_Files/3num_loop_2_aligned_new.pqr")
pqr2.save("./PDB2PQR_Files/3num_loop_2_aligned_new_clean.pdb")

pqr3 = pmd.load_file("./PDB2PQR_Files/3num_loop_3_aligned_new.pqr")
pqr3.save("./PDB2PQR_Files/3num_loop_3_aligned_new_clean.pdb")

## Using OpenEye to Check Bond Orders/Connectivity and Add Hydrogens

In [ ]:
from openeye.oechem import *
from openeye.oeomega import *
from openeye.oeiupac import *
from openeye.oeshape import *

In [ ]:
istream = oemolistream("3num_loop_1_aligned_new_clean.pdb")
mol_from_file = OEMol()
OEReadMolecule(istream, mol_from_file)
oechem.OEAddExplicitHydrogens(OEMol())
oechem.OEDetermineConnectivity(OEMol())
oechem.OEPerceiveBondOrders(OEMol())
ostream = oemolostream('3num_loop_1_aligned_new_clean_oe.pdb')
OEWriteMolecule(ostream, mol_from_file)

In [ ]:
istream = oemolistream("3num_loop_2_aligned_new_clean.pdb")
mol_from_file = OEMol()
OEReadMolecule(istream, mol_from_file)
oechem.OEAddExplicitHydrogens(OEMol())
oechem.OEDetermineConnectivity(OEMol())
oechem.OEPerceiveBondOrders(OEMol())
ostream = oemolostream('3num_loop_2_aligned_new_clean_oe.pdb')
OEWriteMolecule(ostream, mol_from_file)

In [ ]:
istream = oemolistream("3num_loop_3_aligned_new_clean.pdb")
mol_from_file = OEMol()
OEReadMolecule(istream, mol_from_file)
oechem.OEAddExplicitHydrogens(OEMol())
oechem.OEDetermineConnectivity(OEMol())
oechem.OEPerceiveBondOrders(OEMol())
ostream = oemolostream('3num_loop_3_aligned_new_clean_oe.pdb')
OEWriteMolecule(ostream, mol_from_file)

# Needed to add back in header crystal and scale information 

### Loops

![3NUM Loops](./images/3num_loops.png)

Here is a piture of the three starting loop configurations (1: Blue; 2: Red; 3: Orange)

## Calculating the Box Volume

In [3]:
import mdtraj as md
import MDAnalysis as mda

In [4]:
SC = mda.Universe("./3num_loop_1_SC.pdb")

In [8]:
[X, Y, Z, a, b, g ] = SC.dimensions
print("Box Size (breadth, width, height): {} {} {} A".format(X,Y,Z))

Box Size (breadth, width, height): 218.95199584960938 218.95199584960938 227.91200256347656 A


In [7]:
import numpy as np

In [8]:
SCVolume = X * Y * Z * np.sin((g/360.0)*2*np.pi)
print(SCVolume)

9462276.70019


### Calculation of the number of solute molecules

$9,462,276.7 \ \text{A}^{3}\to 9.4622767×10^{-21} \ \text{L}$

$1.0 \ \text{M} \ \text{Li}_{2}^{+} \ \text{SO}_{4}^{2-} \times 9.4622767×10^{-21} \ \text{L} \ \times \ \text{N}_{A}$:
- **5,698 Sulfate molecules, 11,396 Lithium atoms** 
- (3,755 Sulfate molecules, 7,509 Lithum atoms at 65.89% solvent content)

$0.5 \ \text{M} \ (\text{NH}_{4}^{+})_{2} \ \text{SO}_{4}^{2-} \times 9.4622767×10^{-21} \ \text{L} \ \times \ \text{N}_{A}$: 
- **2,849 Sulfate molecules, 5,698 Ammonium molecules** 
- (1,877 Sulfate molecules, 3754 Ammonium molecules at 65.89% solvent content)

$0.1 \ \text{M} \ (\text{Na}^{+})_{2} \ (\text{C}_{6}\text{H}_{6}\text{O}_{7})^{2-} \times 9.4622767×10^{-21} \ \text{L} \ \times \ \text{N}_{A}$: 
- **570 Citrate molecules, 1,140 Sodium molecules** 
- (376 Sulfate molecules, 751 Sodium at 65.89% solvent content)

#### Totals to add: 
Full supercell volume (65.89% solvent content)

- **Sulfate**: 8,547 (5,632)
- **Lithium**: 11,396 (7,510)
- **Sodium**: 1,140 (752)
- **Ammonium**: 5,698 (3754)
- **Citrate**: 570 (376)

## Supercell

In [9]:
SCView = nglview.show_file("./3num_loop_1_SC.pdb")
SCView.add_representation(repr_type="cartoon", selection="protein")
SCView

A Jupyter Widget

## Supercell has a net charge of +144

Need to balance out with negative charged ions (sulfate and citrate):

Need 72 sulfate and citrate ions
Sulfate:Citrate = 5632:376 = 14.9787234043:1
- (basically 15:1 -- groups of 16)
- 72/16 = 4.5
- 4.5 \* 15 = 67.5 --> 68 Sulfate Ions
- 4.5 \* 1  = 4.5  --> 4 Citrate Ions

Add 68 Sulfate ions, 4 Citrate ions

#### New Totals to add: 
Full supercell volume (65.89% solvent content)

- **Sulfate**: 8,615 (5,700)
- **Lithium**: 11,396 (7,510)
- **Sodium**: 1,140 (752)
- **Ammonium**: 5,698 (3754)
- **Citrate**: 574 (380)

**TOTAL**: 27,423 (18,096)

## Adding the Waters

$55.5 \ \text{M} \times \ 9.4622767×10^{-21} \ \text{L} = 316,283  \ \text{molecules} \ \text{(208,399 molecules)}$

Subtracting ions from water to add (65.89%): 208,399-18096 = 190,303

Just going to add 190,300

## Prepping the Solute

In [ ]:
from openeye.oechem import *
from openeye.oeiupac import *
from openeye.oeomega import *
from openeye.oeshape import *
from openeye.oedepict import *

In [ ]:
mol_from_smiles = OEMol()
OEParseSmiles(mol_from_smiles, "[O-]S(=O)(=O)[O-]")

In [ ]:
OEPrepareDepiction(mol_from_smiles)
OERenderMolecule("sulfate.png", mol_from_smiles)

![sulfate](./images/sulfate.png)

For citrate, decided to go with divalent citrate, because the experiment specifies a pH of 5.6 and the pKa specifications from Sigma Aldrich is as follows -- pKa: 3.138, 4.76, 6.401

In [ ]:
mol_from_smiles = OEMol()
OEParseSmiles(mol_from_smiles, "C(C(=O)[O-])C(CC(=O)[O-])(C(=O)O)O")
OEPrepareDepiction(mol_from_smiles)
OERenderMolecule("citrate_2minus.png", mol_from_smiles)

![citrate_2minus](./images/citrate_2minus.png)

In [ ]:
mol_from_smiles = OEMol()
OEParseSmiles(mol_from_smiles, "[NH4+]")
OEPrepareDepiction(mol_from_smiles)
OERenderMolecule("ammonium.png", mol_from_smiles)

![ammonium](./images/ammonium.png)

#### Code used to create all of the solute pdbs (replacing all the strings in SmilesToMol with the respective SMILES strings, and the string in oemolostream):

```
from openeye.oechem import *
from openeye.oeomega import *
from openeye.oeiupac import *
from openeye.oeshape import *
from openeye.oequacpac import *

mol = OEMol()
OESmilesToMol(mol, "[NH4+]")
#OEAddExplicitHydrogens(mol)

mol.SetTitle("Ammonium")

omega = OEOmega()
omega.SetMaxConfs(1)
omega.SetStrictStereo(False)
omega.SetStrictAtomTypes(False)
omega(mol)
OEDetermineConnectivity(mol)
OEAssignCharges(mol, OEAM1BCCCharges())

ostream = oemolostream("ammonium.mol2")
OEWriteMolecule(ostream, mol)
ostream = oemolostream("ammonium.pdb")
OEWriteMolecule(ostream, mol)
```

### Viewing them

In [1]:
import nglview

view1 = nglview.show_file("/Users/davidwych/Downloads/GP3_Dir/ammonium.mol2")
view2 = nglview.show_file("/Users/davidwych/Downloads/GP3_Dir/citrate.mol2")
view3 = nglview.show_file("/Users/davidwych/Downloads/GP3_Dir/sulfate.mol2")

In [2]:
from ipywidgets import VBox
vbox = VBox([view1, view2, view3])
vbox

A Jupyter Widget

### Using antechamber to give molecules gaff charges with commands:

`antechamber -i sulfate.mol2 -fi mol2 -o sulfate.gaff.mol2 -fo mol2 -c bcc -nc -2`

```
acdoctor mode is on: check and diagnosis problems in the input file.
-- Check Format for mol2 File --
   Status: pass
-- Check Unusual Elements --
   Status: pass
-- Check Open Valences --
Warning: This molecule has no hydrogens nor halogens.
         It is quite possible that there are unfilled valences.
Warning: The number of bonds (1) for atom (ID: 1, Name: O1) does not match
         the connectivity (2) for atom type (O.3) defined in CORR_NAME_TYPE.DAT.
Warning: The number of bonds (1) for atom (ID: 2, Name: O2) does not match
         the connectivity (2) for atom type (O.3) defined in CORR_NAME_TYPE.DAT.
But, you may safely ignore the warnings if your molecule
         uses atom names or element names as atom types.
-- Check Geometry --
      for those bonded
      for those not bonded
   Status: pass
-- Check Weird Bonds --
   Status: pass
-- Check Number of Units --
   Status: pass
acdoctor mode has completed checking the input file.

Info: Bond types are assigned for valence state (1) with penalty (1).
Info: Total number of electrons: 50; net charge: -2

Running: /Users/davidwych/anaconda3/bin/sqm -O -i sqm.in -o sqm.out 
```

`antechamber -i citrate.mol2 -fi mol2 -o citrate.gaff.mol2 -fo mol2 -c bcc -nc -2`

```
acdoctor mode is on: check and diagnosis problems in the input file.
-- Check Format for mol2 File --
   Status: pass
-- Check Unusual Elements --
   Status: pass
-- Check Open Valences --
   Status: pass
-- Check Geometry --
      for those bonded
      for those not bonded
   Status: pass
-- Check Weird Bonds --
   Status: pass
-- Check Number of Units --
   Status: pass
acdoctor mode has completed checking the input file.

Info: Total number of electrons: 100; net charge: -2

Running: /Users/davidwych/anaconda3/bin/sqm -O -i sqm.in -o sqm.out
```

`antechamber -i ammonium.mol2 -fi mol2 -o ammonium.gaff.mol2 -fo mol2 -c bcc -nc +1`

```
acdoctor mode is on: check and diagnosis problems in the input file.
-- Check Format for mol2 File --
   Status: pass
-- Check Unusual Elements --
   Status: pass
-- Check Open Valences --
   Status: pass
-- Check Geometry --
      for those bonded
      for those not bonded
   Status: pass
-- Check Weird Bonds --
   Status: pass
-- Check Number of Units --
   Status: pass
acdoctor mode has completed checking the input file.

Info: Total number of electrons: 10; net charge: 1

Running: /Users/davidwych/anaconda3/bin/sqm -O -i sqm.in -o sqm.out
```

Then used antechamber to convert gaff.mol2 files to pdb files:
    
`antechamber -i ammonium.gaff.mol2 -fi mol2 -o ammonium.pdb -fo pdb`


```
acdoctor mode is on: check and diagnosis problems in the input file.
-- Check Format for mol2 File --
   Status: pass
-- Check Unusual Elements --
   Status: pass
-- Check Open Valences --
   Status: pass
-- Check Geometry --
      for those bonded
      for those not bonded
   Status: pass
-- Check Weird Bonds --
   Status: pass
-- Check Number of Units --
   Status: pass
acdoctor mode has completed checking the input file.
```

`antechamber -i citrate.gaff.mol2 -fi mol2 -o citrate.pdb -fo pdb`

```
acdoctor mode is on: check and diagnosis problems in the input file.
-- Check Format for mol2 File --
   Status: pass
-- Check Unusual Elements --
   Status: pass
-- Check Open Valences --
   Status: pass
-- Check Geometry --
      for those bonded
      for those not bonded
   Status: pass
-- Check Weird Bonds --
/Users/davidwych/anaconda3/bin/to_be_dispatched/antechamber: Fatal Error!
```
`antechamber -i sulfate.gaff.mol2 -fi mol2 -o sulfate.pdb -fo pdb`

```
acdoctor mode is on: check and diagnosis problems in the input file.
-- Check Format for mol2 File --
   Status: pass
-- Check Unusual Elements --
   Status: pass
-- Check Open Valences --
Warning: This molecule has no hydrogens nor halogens.
         It is quite possible that there are unfilled valences.
-- Check Geometry --
      for those bonded
      for those not bonded
   Status: pass
-- Check Weird Bonds --
   Status: pass
-- Check Number of Units --
   Status: pass
acdoctor mode has completed checking the input file.
```